script to preprocess the poll data based on the R script that I created
```
process.dates <- function(input.date.vect){
  input.date.vect <- gsub(".{3,} - ", "", input.date.vect)
  current.month <- rep(NA, length(input.date.vect))
  #need to add years to the vector
  current.year <- 2016
  #identify month
  for(i in 1:length(input.date.vect)){
    #if the second character is a slash, then the month is one digit. else, two
    second.char <- substr(input.date.vect[i], 2, 2)
    if(second.char == "/"){
      current.month[i] = substr(input.date.vect[i], 1, 1)
    } else {
      current.month[i] = substr(input.date.vect[i], 1, 2)
    }
    # if the month of the previous entry is greater, then knock down the year one
    if(i > 1){
      if(current.month[i] > current.month[i - 1]){
        current.year <- current.year - 1
      }
    }
    input.date.vect[i] = paste0(input.date.vect[i], "/", current.year)
  }
  input.date.vect <- gsub("/", "-", input.date.vect)
  #add zeros where necessary
  input.date.vect <- sub("^([0-9]{1})-", "0\\1-", input.date.vect)
  input.date.vect <- sub("-([0-9]{1})-", "-0\\1-", input.date.vect)
  return(input.date.vect)
}

reps$Date <- as.POSIXct(as.Date(process.dates(reps$Date), "%m-%d-%Y"))
dems$Date <- as.POSIXct(as.Date(process.dates(dems$Date), "%m-%d-%Y"))

min.clinton <- min(dems$Date[dems$Clinton != "--"])
max.clinton <- max(dems$Date[dems$Clinton != "--"])
min.bernie <- min(dems$Date[dems$Sanders != "--"])
max.bernie <- max(dems$Date[dems$Sanders != "--"])
min.trump <- min(reps$Date[reps$Trump != "--"])
max.trump <- max(reps$Date[reps$Trump != "--"])
min.cruz <- min(reps$Date[reps$Cruz != "--"])
max.cruz <- max(reps$Date[reps$Cruz != "--"])
min.kasich <- min(reps$Date[reps$Kasich != "--"])
max.kasich <- max(reps$Date[reps$Kasich != "--"])

len.clinton <- max.clinton - min.clinton
len.bernie <- max.bernie - min.bernie
len.trump <- max.trump - min.trump
len.cruz <- max.cruz - min.cruz
len.kasich <- max.kasich - min.kasich

candidate.data.frame <- data.frame(campaign.length = as.numeric(c(len.clinton, len.bernie, len.trump, len.cruz, len.kasich)))
rownames(candidate.data.frame) <- c("Clinton", "Sanders", "Trump", "Cruz", "Kasich")
# take the lyric data and add the mentions per candidate for data to create a scatterplot

candidate.data.frame$mentions <- NA
candidate.data.frame$mentions[1] <- sum(lyrics$candidate == "Hillary Clinton")
candidate.data.frame$mentions[2] <- sum(lyrics$candidate == "Bernie Sanders")
candidate.data.frame$mentions[3] <- sum(lyrics$candidate == "Donald Trump")
candidate.data.frame$mentions[4] <- sum(lyrics$candidate == "Ted Cruz")
candidate.data.frame$mentions[5] <- sum(lyrics$candidate == "John Kasich")
```

In [116]:
# author: John Boudreaux

import numpy as np
import pandas as pd
import re


dems_2016 = pd.read_csv("../data/democrats2016.csv")
reps_2016 = pd.read_csv("../data/republicans2016.csv")
dems = dems_2016.copy()


def get_current_month(string_val):
    ''' for date objects, will retreive the first or second value before a slash'''
    if string_val[1] == "/":
        return(int(string_val[0:1]))
    else:
        return(int(string_val[0:2]))
    
def get_day(string_val):
    ''' for preprocessed dates, retreive the day (comes after month and slash)'''
    # find where the slash is
    slash_index = string_val.index("/")
    return(int(string_val[slash_index+1:]))
    
    
 

def fix_dates(old_df):
    ''' function to preprocess the 2016 poll data to give usable datetime objects'''
    df = old_df
    df["preprocessDate"] = df.Date.str.replace(".{3,} - ", "")
    df["month"] = df.preprocessDate.apply(lambda x : get_current_month(x))
    year = 2016
    years = []
    # we need to go and loop through each of the months to figure out which year we need to use
    # if the next month is greater than the previous one, we know we went down a year
    for i in range(len(df.month)):
        if i > 0:
            if df.month[i] > df.month[i-1]:
                year -= 1
        years.append(year)
    df["year"] = years
    df["day"] = df.preprocessDate.apply(lambda x : get_day(x))
    df["newDate"] = pd.to_datetime(df[['day', 'month', 'year']])
    
    columns = ['Date', 'preprocessDate', 'month', 'day', 'year']
    df.drop(columns, inplace=True, axis=1)

    return(df)

def replace_dashes(df):
    df.replace('--', np.nan, inplace = True)
    return(df)

def preprocess_poll_data(df):
    new_df = df
    new_df = fix_dates(new_df)
    new_df = replace_dashes(new_df)
    return(new_df)

new_dems = preprocess_poll_data(dems_2016)
new_reps = preprocess_poll_data(reps_2016)

new_dems.to_csv("../data/PreProcessedDemPolls.csv")
new_reps.to_csv("../data/PreProcessedRepPolls.csv")

### Sanity checks
__Democratic polling data__
* Since we are only using this for the dates that each candidate was in the running in the primaries, we will focus primarily on the dates and polling numbers for the candidates

In [117]:
new_dems.head()

,Poll,Sample,MoE,Clinton,Sanders,Spread,newDate
0,RCP Average,NaN,NaN,52.8,41.4,Clinton +11.4,2016-06-05
1,IBD/TIPP,351 RV,5.3,51.0,37,Clinton +14,2016-06-05
2,Quinnipiac,678 RV,3.8,53.0,39,Clinton +14,2016-05-30
3,ABC News/Wash Post,373 RV,NaN,56.0,42,Clinton +14,2016-05-19
4,NBC News/Wall St. Jrnl,342 LV,5.3,53.0,45,Clinton +8,2016-05-19


In [118]:
# the RCP average is going to throw off our results... since this is the overall average of all polls
# need to take it out
new_dems.drop(new_dems.index[0], inplace = True)
new_dems.reset_index()
new_dems.head()

,Poll,Sample,MoE,Clinton,Sanders,Spread,newDate
1,IBD/TIPP,351 RV,5.3,51.0,37,Clinton +14,2016-06-05
2,Quinnipiac,678 RV,3.8,53.0,39,Clinton +14,2016-05-30
3,ABC News/Wash Post,373 RV,NaN,56.0,42,Clinton +14,2016-05-19
4,NBC News/Wall St. Jrnl,342 LV,5.3,53.0,45,Clinton +8,2016-05-19
5,CBS News/NY Times,371 LV,6,51.0,44,Clinton +7,2016-05-17


In [119]:
new_dems.shape[0] # this is how many rows we have...

167

In [120]:
# check the Clinton column
new_dems.Clinton.describe()
#seems ok, no negative vals or vals greater than 100

count    167.000000
mean      55.287425
std        7.780189
min       33.000000
25%       50.000000
50%       55.000000
75%       61.000000
max       75.000000
Name: Clinton, dtype: float64

In [121]:
new_dems.Clinton.isnull().sum()

0

In [122]:
# check Sanders column
new_dems.Sanders.describe()

count     138
unique     48
top         4
freq        7
Name: Sanders, dtype: object

In [123]:
new_dems.Sanders.isnull().sum()

29

In [124]:
new_dems.newDate[new_dems.Sanders.isnull()]
# in general, sanders wasn't in the race before 11/2014 is what we will say

136   2014-11-21
139   2014-07-22
140   2014-07-20
141   2014-06-30
143   2014-04-15
144   2014-03-30
145   2014-03-09
146   2014-03-09
147   2014-01-26
148   2014-01-23
149   2014-01-19
150   2013-12-16
151   2013-12-15
152   2013-12-09
153   2013-12-05
154   2013-11-20
155   2013-11-08
156   2013-10-31
157   2013-09-29
158   2013-09-08
159   2013-08-02
160   2013-07-21
161   2013-07-18
162   2013-05-09
163   2013-04-29
164   2013-03-30
165   2013-02-03
166   2013-01-06
167   2012-12-02
Name: newDate, dtype: datetime64[ns]

In [125]:
# seems like pandas thinks this is a categorical variable.
pd.to_numeric(new_dems.Sanders).describe()
# this seems much better. count is less than 168 which we expect b/c he was not in race as long
# no vals above 100, below 0

count    138.000000
mean      26.246377
std       13.908886
min        1.000000
25%       15.000000
50%       29.000000
75%       37.750000
max       49.000000
Name: Sanders, dtype: float64

In [126]:
# check on the newDate column
new_dems.newDate.describe()
# again, this is treated as a factor.. let's look at head/tail

count                     167
unique                    139
top       2015-10-18 00:00:00
freq                        3
first     2012-12-02 00:00:00
last      2016-06-05 00:00:00
Name: newDate, dtype: object

In [127]:
new_dems.newDate.tail()

163   2013-04-29
164   2013-03-30
165   2013-02-03
166   2013-01-06
167   2012-12-02
Name: newDate, dtype: datetime64[ns]

In [128]:
new_dems.newDate.head()

1   2016-06-05
2   2016-05-30
3   2016-05-19
4   2016-05-19
5   2016-05-17
Name: newDate, dtype: datetime64[ns]

In [129]:
new_dems.newDate.isnull().sum()

0

__Republican polling data__
* Since we are only using this for the dates that each candidate was in the running in the primaries, we will focus primarily on the dates and polling numbers for the candidates

In [130]:
new_reps.head()

,Poll,Sample,Trump,Cruz,Kasich,Spread,newDate
0,RCP Average,NaN,46.5,27,18,Trump +19.5,2016-05-01
1,CNN/ORC,406 LV,49,25,19,Trump +24,2016-05-01
2,IBD/TIPP,397 RV,48,29,16,Trump +19,2016-04-28
3,USA Today/Suffolk,292 LV,45,29,17,Trump +16,2016-04-24
4,Pew Research,740 RV,44,25,20,Trump +19,2016-04-19


In [131]:
#need to take out RCP average for same reasons as dems data
new_reps.drop(new_reps.index[0], inplace = True)
new_reps.reset_index()
new_reps.head()

,Poll,Sample,Trump,Cruz,Kasich,Spread,newDate
1,CNN/ORC,406 LV,49,25,19,Trump +24,2016-05-01
2,IBD/TIPP,397 RV,48,29,16,Trump +19,2016-04-28
3,USA Today/Suffolk,292 LV,45,29,17,Trump +16,2016-04-24
4,Pew Research,740 RV,44,25,20,Trump +19,2016-04-19
5,FOX News,419 RV,45,27,25,Trump +18,2016-04-13


In [132]:
new_reps.shape[0]

169

In [133]:
new_reps.tail(10)

,Poll,Sample,Trump,Cruz,Kasich,Spread,newDate
160,CNN/ORC,452 A,NaN,7,NaN,Christie +4,2013-09-08
161,Rasmussen Reports,LV,NaN,NaN,NaN,Christie +3,2013-08-02
162,PPP (D),500 RV,NaN,12,NaN,Paul +3,2013-07-21
163,McClatchy/Marist,357 RV,NaN,7,NaN,Christie +3,2013-07-18
164,PPP (D),806 RV,NaN,7,NaN,Rubio +1,2013-05-09
165,Quinnipiac,712 RV,NaN,NaN,NaN,Rubio +4,2013-04-01
166,PPP (D),1125 RV,NaN,NaN,NaN,Rubio +4,2013-03-30
167,PPP (D),508 RV,NaN,NaN,NaN,Rubio +9,2013-02-03
168,PPP (D),563 RV,NaN,NaN,NaN,Rubio +6,2013-01-06
169,PPP (D),475 RV,NaN,NaN,NaN,Rubio +4,2012-12-02


In [134]:
new_reps.apply(lambda x : x.isnull().sum())

Poll        0
Sample      1
Trump      58
Cruz        7
Kasich     38
Spread      0
newDate     0
dtype: int64

In [135]:
new_reps.newDate[new_reps.Trump.isnull()]

110   2015-05-10
111   2015-04-30
113   2015-04-21
114   2015-04-19
116   2015-03-29
117   2015-03-31
118   2015-03-15
119   2015-03-04
120   2015-03-02
121   2015-02-22
122   2015-02-15
123   2015-01-27
124   2014-12-21
125   2014-12-14
126   2014-12-09
127   2014-11-23
128   2014-11-23
129   2014-11-21
130   2014-10-12
131   2014-09-29
132   2014-08-04
133   2014-07-22
134   2014-07-20
135   2014-06-30
136   2014-06-01
137   2014-05-04
138   2014-04-27
139   2014-04-15
140   2014-04-10
141   2014-03-30
142   2014-03-20
143   2014-03-09
144   2014-03-09
145   2014-02-09
146   2014-02-02
147   2014-01-26
148   2014-01-23
149   2014-01-19
150   2014-01-14
151   2013-12-16
152   2013-12-15
153   2013-12-09
154   2013-12-05
155   2013-11-20
156   2013-11-08
157   2013-10-31
158   2013-09-29
159   2013-09-26
160   2013-09-08
161   2013-08-02
162   2013-07-21
163   2013-07-18
164   2013-05-09
165   2013-04-01
166   2013-03-30
167   2013-02-03
168   2013-01-06
169   2012-12-02
Name: newDate,

In [136]:
new_reps.newDate[new_reps.Kasich.isnull()]

110   2015-05-10
111   2015-04-30
117   2015-03-31
119   2015-03-04
121   2015-02-22
129   2014-11-21
131   2014-09-29
132   2014-08-04
134   2014-07-20
136   2014-06-01
137   2014-05-04
139   2014-04-15
141   2014-03-30
142   2014-03-20
143   2014-03-09
144   2014-03-09
146   2014-02-02
147   2014-01-26
148   2014-01-23
150   2014-01-14
151   2013-12-16
152   2013-12-15
154   2013-12-05
155   2013-11-20
156   2013-11-08
157   2013-10-31
158   2013-09-29
159   2013-09-26
160   2013-09-08
161   2013-08-02
162   2013-07-21
163   2013-07-18
164   2013-05-09
165   2013-04-01
166   2013-03-30
167   2013-02-03
168   2013-01-06
169   2012-12-02
Name: newDate, dtype: datetime64[ns]

In [137]:
new_reps.newDate[new_reps.Cruz.isnull()]

129   2014-11-21
161   2013-08-02
165   2013-04-01
166   2013-03-30
167   2013-02-03
168   2013-01-06
169   2012-12-02
Name: newDate, dtype: datetime64[ns]

In [138]:
new_reps.Trump.astype("float64").describe()

count    111.000000
mean      28.495495
std       11.229728
min        1.000000
25%       24.000000
50%       28.000000
75%       36.000000
max       49.000000
Name: Trump, dtype: float64

In [139]:
new_reps.Kasich.astype("float64").describe()

count    131.000000
mean       5.633588
std        6.535355
min        0.000000
25%        2.000000
50%        3.000000
75%        5.000000
max       25.000000
Name: Kasich, dtype: float64

In [140]:
new_reps.Cruz.astype("float64").describe()

count    162.000000
mean      12.740741
std        8.460765
min        3.000000
25%        6.000000
50%        9.000000
75%       18.000000
max       38.000000
Name: Cruz, dtype: float64

In [141]:
(new_reps.newDate[1] - max(new_reps.newDate[new_reps.Cruz.isnull()])).components.days


527

### Calculate Campaign Length, up until national convention


In [142]:
dict_of_lengths = {'Trump' : (new_reps.newDate[1] - max(new_reps.newDate[new_reps.Trump.isnull()])).components.days, \
                   'Cruz' : (new_reps.newDate[1] - max(new_reps.newDate[new_reps.Cruz.isnull()])).components.days, \
                   'Kasich' : (new_reps.newDate[1] - max(new_reps.newDate[new_reps.Trump.isnull()])).components.days, \
                   'Clinton' : (new_dems.newDate[1] - min(new_dems.newDate)).components.days, \
                   'Sanders' : (new_dems.newDate[1] - max(new_dems.newDate[new_dems.Sanders.isnull()])).components.days
                  }
campaign_length = pd.DataFrame.from_dict(dict_of_lengths, orient = 'index').transpose()
campaign_length.to_csv("../data/campaign_length.csv")